In [13]:
from pyspark.sql import SparkSession
#Initialize a PySpark SparkSession
spark = SparkSession.builder.appName("lr_PySpark").getOrCreate()

In [14]:
data = spark.read.format("csv").option("header", "true").load("./ml_model/fraudulent_transactions_anonymous_ia.csv")

In [3]:
data.printSchema()

root
 |-- transaction-id: string (nullable = true)
 |-- customer-id: string (nullable = true)
 |-- account-id: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- customer_id_avrge_amount_1day: string (nullable = true)
 |-- customer_id_avrge_amount_1week: string (nullable = true)
 |-- customer_id_avrge_amount_1month: string (nullable = true)
 |-- customer_id_avrge_amount_3month: string (nullable = true)
 |-- customer_id_count_1day: string (nullable = true)
 |-- customer_id_count_1week: string (nullable = true)
 |-- customer_id_count_1month: string (nullable = true)
 |-- customer_id_count_3month: string (nullable = true)
 |-- account_id_avrge_amount_1day: string (nullable = true)
 |-- account_id_avrge_amount_1week: string (nullable = true)
 |-- account_id_avrge_amount_1month: string (nullable = true)
 |-- account_id_avrge_amount_3month: string (nullable = true)
 |-- account_id_count_1day: string (nullable = true)
 |-- account_id_co

In [15]:
from pyspark.sql.functions import col

# Convert  column from String to Double
data = data.withColumn("amount", col("amount").cast("double"))

data = data.withColumn("customer_id_avrge_amount_1day", col("customer_id_avrge_amount_1day").cast("double"))
data = data.withColumn("customer_id_avrge_amount_1week", col("customer_id_avrge_amount_1week").cast("double"))
data = data.withColumn("customer_id_avrge_amount_1month", col("customer_id_avrge_amount_1month").cast("double"))
data = data.withColumn("customer_id_avrge_amount_3month", col("customer_id_avrge_amount_3month").cast("double"))

data = data.withColumn("customer_id_count_1day", col("customer_id_count_1day").cast("integer"))
data = data.withColumn("customer_id_count_1week", col("customer_id_count_1week").cast("integer"))
data = data.withColumn("customer_id_count_1month", col("customer_id_count_1month").cast("integer"))
data = data.withColumn("customer_id_count_3month", col("customer_id_count_3month").cast("integer"))

data = data.withColumn("account_id_avrge_amount_1day", col("account_id_avrge_amount_1day").cast("double"))
data = data.withColumn("account_id_avrge_amount_1week", col("account_id_avrge_amount_1week").cast("double"))
data = data.withColumn("account_id_avrge_amount_1month", col("account_id_avrge_amount_1month").cast("double"))
data = data.withColumn("account_id_avrge_amount_3month", col("account_id_avrge_amount_3month").cast("double"))

data = data.withColumn("account_id_count_1day", col("account_id_count_1day").cast("integer"))
data = data.withColumn("account_id_count_1week", col("account_id_count_1week").cast("integer"))
data = data.withColumn("account_id_count_1month", col("account_id_count_1month").cast("integer"))
data = data.withColumn("account_id_count_3month", col("account_id_count_3month").cast("integer"))

data = data.withColumn("transaction_in_weekend", col("transaction_in_weekend").cast("integer"))

data = data.withColumn("transaction_at_night", col("transaction_at_night").cast("integer"))

data = data.withColumn("is_fraud", col("is_fraud").cast("integer"))

In [5]:
data.printSchema()

root
 |-- transaction-id: string (nullable = true)
 |-- customer-id: string (nullable = true)
 |-- account-id: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- customer_id_avrge_amount_1day: double (nullable = true)
 |-- customer_id_avrge_amount_1week: double (nullable = true)
 |-- customer_id_avrge_amount_1month: double (nullable = true)
 |-- customer_id_avrge_amount_3month: double (nullable = true)
 |-- customer_id_count_1day: integer (nullable = true)
 |-- customer_id_count_1week: integer (nullable = true)
 |-- customer_id_count_1month: integer (nullable = true)
 |-- customer_id_count_3month: integer (nullable = true)
 |-- account_id_avrge_amount_1day: double (nullable = true)
 |-- account_id_avrge_amount_1week: double (nullable = true)
 |-- account_id_avrge_amount_1month: double (nullable = true)
 |-- account_id_avrge_amount_3month: double (nullable = true)
 |-- account_id_count_1day: integer (nullable = true)
 |-- account_

In [16]:
%%time

from pyspark.ml.feature import VectorAssembler

# Define the input and output features
output_feature = "is_fraud"
input_features = [
    'amount',
    'customer_id_avrge_amount_1day', 'customer_id_avrge_amount_1week', 'customer_id_avrge_amount_1month', 'customer_id_avrge_amount_3month',
    'customer_id_count_1day', 'customer_id_count_1week', 'customer_id_count_1month', 'customer_id_count_3month',
    'account_id_avrge_amount_1day', 'account_id_avrge_amount_1week', 'account_id_avrge_amount_1month', 'account_id_avrge_amount_3month',
    'account_id_count_1day', 'account_id_count_1week', 'account_id_count_1month', 'account_id_count_3month',
    'transaction_in_weekend', 'transaction_at_night'
]

# Create a VectorAssembler for combining input features into a feature vector
assembler = VectorAssembler(inputCols=input_features, outputCol="features")

df_data=assembler.transform(data)

model_df=df_data.select(['features','is_fraud'])

# Split the data into training and test sets
(trainingData, testData) = model_df.randomSplit([0.7, 0.3])

CPU times: user 60.6 ms, sys: 4.83 ms, total: 65.4 ms
Wall time: 3.73 s


In [12]:
testData.show()

+--------------------+--------+
|            features|is_fraud|
+--------------------+--------+
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
|[3.194,3.194,3.19...|       0|
+--------------------+--------+
only showing top 20 rows



In [17]:
%%time

from pyspark.ml.classification import LogisticRegression

log_reg=LogisticRegression(labelCol=output_feature).fit(trainingData)

# training results
train_results=log_reg.evaluate(trainingData).predictions

CPU times: user 177 ms, sys: 144 ms, total: 322 ms
Wall time: 1min 55s


In [18]:
%%time

from pyspark.sql.functions import expr

# Counting the total number of instances
total_count = train_results.count()

# Counting the number of instances where prediction matches is_fraud
matching_count = train_results.filter(expr("prediction = is_fraud")).count()

# Calculating the percentage
matching_percentage = (matching_count / total_count) * 100

print("Percentage of instances where prediction matches is_fraud:", matching_percentage)

Percentage of instances where prediction matches is_fraud: 94.51415359512971
CPU times: user 23.3 ms, sys: 14.4 ms, total: 37.7 ms
Wall time: 1min 4s


In [19]:
%%time

test_results=log_reg.evaluate(testData).predictions

CPU times: user 4.71 ms, sys: 5.09 ms, total: 9.79 ms
Wall time: 1.38 s


In [20]:
%%time

from pyspark.sql.functions import expr

# Counting the total number of instances
total_count = test_results.count()

# Counting the number of instances where prediction matches is_fraud
matching_count = test_results.filter(expr("prediction = is_fraud")).count()

# Calculating the percentage
matching_percentage = (matching_count / total_count) * 100

print("Percentage of instances where prediction matches is_fraud:", matching_percentage)

Percentage of instances where prediction matches is_fraud: 94.45784770124706
CPU times: user 62.9 ms, sys: 25 ms, total: 87.9 ms
Wall time: 1min 3s


In [21]:
%%time

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Creating a MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction")

# Calculating precision
precision = evaluator.evaluate(test_results, {evaluator.metricName: "weightedPrecision"})
print("Precision:", precision)

# Calculating recall
recall = evaluator.evaluate(test_results, {evaluator.metricName: "weightedRecall"})
print("Recall:", recall)

# Calculating accuracy
accuracy = evaluator.evaluate(test_results, {evaluator.metricName: "accuracy"})
print("Accuracy:", accuracy)

# Calculating the F1 score
f1_score = evaluator.evaluate(test_results, {evaluator.metricName: "f1"})

print("F1 Score:", f1_score)

Precision: 0.9434809334939014
Recall: 0.9445784770124707
Accuracy: 0.9445784770124707
F1 Score: 0.9439198677888532
CPU times: user 363 ms, sys: 282 ms, total: 645 ms
Wall time: 2min 5s


In [22]:
# Save the model
modelPath = "lr_ml"
log_reg.save(modelPath)